In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import datetime

In [54]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
# Import the data
purchase_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/purchase.csv")
boxes_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/boxes.csv")
problem_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/problem 3.csv")

purchase_df



#len(pd.unique(purchase_df['BOX_COUNT']))
#len(pd.unique(boxes_df['BOX_ID']))

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT
0,1/2/2019,2CED678A247,12.0,1.0
1,1/2/2019,2BF58D91BA1,12.0,1.0
2,1/2/2019,2C15B86534E,99.0,1.0
3,1/2/2019,2C32D9A859A,6.0,1.0
4,1/2/2019,2C7A55404D1,4.0,1.0
...,...,...,...,...
2455859,28/10/2018,2BD992B5538,12.0,1.0
2455860,28/10/2018,2C97CD72233,17.0,1.0
2455861,28/10/2018,2C91C61D372,40.0,1.0
2455862,28/10/2018,2CD70CFC4E3,51.0,1.0


In [64]:
max_box_id = purchase_df['PURCHASE_DATE'].max()

min_box_id = purchase_df['PURCHASE_DATE'].min()

print(max_box_id)
print(min_box_id)

2019-02-28 00:00:00
2018-10-01 00:00:00


In [56]:

max_box_id = boxes_df['BOX_ID'].max()

min_box_id = boxes_df['BOX_ID'].min()

print(max_box_id)
print(min_box_id)

290
1


In [57]:
#Removing all the nan and zero BOX COUNT Values

#Remove for -1 values
purchase_df = purchase_df[purchase_df['BOX_COUNT'] >= 0]

#Remove for nan values
purchase_df = purchase_df.dropna(subset=['BOX_COUNT'])



#Drop Box id which is not present in boxes
indexAge = purchase_df[ (purchase_df['BOX_ID'] < min_box_id) | (purchase_df['BOX_ID'] > max_box_id) ].index
purchase_df.drop(indexAge , inplace=True)

#Drop Date which Not needed
# indexAge = purchase_df[ (purchase_df['PURCHASE_DATE'] <"2019-03-01") | (purchase_df['PURCHASE_DATE'] >"2019-03-15") ].index
# purchase_df.drop(indexAge , inplace=True)



#Changing the type of the columns
purchase_df['PURCHASE_DATE'] = pd.to_datetime(purchase_df['PURCHASE_DATE'], format='%d/%m/%Y')
purchase_df['MAGIC_KEY'] = purchase_df['MAGIC_KEY'].astype('string')
purchase_df['BOX_ID'] = purchase_df['BOX_ID'].astype('int')
purchase_df['BOX_COUNT'] = purchase_df['BOX_COUNT'].astype('int')



purchase_df.dtypes


PURCHASE_DATE    datetime64[ns]
MAGIC_KEY        string[python]
BOX_ID                    int64
BOX_COUNT                 int64
dtype: object

In [58]:
#Merging both the datasets
df_master = purchase_df.merge(boxes_df)


In [59]:
df_master = df_master.assign(TOTAL_PRICE = lambda x: (x['BOX_COUNT'] * x['UNIT_PRICE']))
df_master['PURCHASE_DATE'] = pd.to_datetime(df_master['PURCHASE_DATE'], dayfirst = True)

df_master

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,TOTAL_PRICE
0,2019-02-01,2CED678A247,12,1,Premium,Home Delivery - CoD,8.0,1.5,12.98,12.98
1,2019-02-01,2BF58D91BA1,12,1,Premium,Home Delivery - CoD,8.0,1.5,12.98,12.98
2,2019-02-01,2C3A3F7DB65,12,1,Premium,Home Delivery - CoD,8.0,1.5,12.98,12.98
3,2019-02-01,2BD2641961A,12,1,Premium,Home Delivery - CoD,8.0,1.5,12.98,12.98
4,2019-02-01,2BD2EAA1C90,12,1,Premium,Home Delivery - CoD,8.0,1.5,12.98,12.98
...,...,...,...,...,...,...,...,...,...,...
2455795,2018-10-14,2BE6A6A7E6C,169,1,Premium,Delivery from Collection Point,14.0,2.9,23.98,23.98
2455796,2018-10-14,2C774DAC048,169,1,Premium,Delivery from Collection Point,14.0,2.9,23.98,23.98
2455797,2018-10-14,2BD1A4A1DC5,169,1,Premium,Delivery from Collection Point,14.0,2.9,23.98,23.98
2455798,2018-10-13,28D8EFAE836,264,2,Standard,Delivery from Collection Point,13.0,2.2,17.98,35.96


In [47]:
#Drop Magic Key which is not in the problem


index = df_master[ ((df_master['PURCHASE_DATE'] < "2019-03-01") | (df_master['PURCHASE_DATE'] >"2019-03-15"))].index
df_master.drop(index , inplace=True)


df_master

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,TOTAL_PRICE


In [60]:
df_final = df_master.merge(problem_df)

df_final

,PURCHASE_DATE,MAGIC_KEY,BOX_ID,BOX_COUNT,QUALITY,DELIVERY_OPTION,MILK,MEAT,UNIT_PRICE,TOTAL_PRICE
0,2019-02-01,2CBA39D1B2B,12,1,Premium,Home Delivery - CoD,8.0,1.5,12.98,12.98
1,2018-12-13,2CBA39D1B2B,134,1,Premium,Delivery from Collection Point,10.0,1.8,19.98,19.98
2,2018-11-04,2CBA39D1B2B,134,1,Premium,Delivery from Collection Point,10.0,1.8,19.98,19.98
3,2018-10-05,2CBA39D1B2B,137,1,Premium,Delivery from Collection Point,12.0,1.8,19.98,19.98
4,2019-02-11,2BEA50C143F,12,1,Premium,Home Delivery - CoD,8.0,1.5,12.98,12.98
...,...,...,...,...,...,...,...,...,...,...
15348,2018-10-05,2BD0B583688,129,1,Premium,Delivery from Collection Point,12.0,2.2,15.98,15.98
15349,2018-10-11,28F62D28A57,260,1,Standard,Delivery from Collection Point,10.0,1.8,17.98,17.98
15350,2018-10-12,297CED65B9E,260,1,Standard,Delivery from Collection Point,10.0,1.8,17.98,17.98
15351,2018-10-14,2BDFE58E38B,34,1,Premium,Home Delivery - CoD,10.0,2.0,15.98,15.98


In [61]:
from tqdm import tqdm

df_answer = pd.DataFrame(columns=['MAGIC_KEY', 'MEAT'])

#df_answer.dtypes

df_answer['MAGIC_KEY'] = df_final['MAGIC_KEY']
df_answer['MEAT'] = df_final['MEAT']

#df_answer = df_answer.assign(BOX_ID = 0)

# df2 = df.assign(Tutors='NAN')
#df_answer

problem_df = problem_df.assign(MEAT = 0)

problem_df

for i in tqdm(range(len(problem_df))):

  val = problem_df.iloc[i, 0]
  #print(val)
  if val in df_answer['MAGIC_KEY'].values:

    #print(id)
    res_df =  df_answer.loc[df_answer['MAGIC_KEY'] == val]
    s = res_df['MEAT']

    counts = s.value_counts()
    # print the most frequent value
    #print(counts.index[0])
    problem_df.iloc[i, 1] = counts.index[0]


#problem_df

100%|██████████| 5379/5379 [00:20<00:00, 262.63it/s]


In [62]:
problem_df.to_csv('sample submission 3.csv', index=False)